In [1]:
import os
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [233]:
pwdf = pd.read_csv('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-10-18_Compare_rush/2020-10-18_Compare_rush.snp.pairwise.tsv', sep="\t" , names=['sample_1', 'sample_2', 'dist'])

In [234]:
pwdf.head()

,sample_1,sample_2,dist
0,20069944,20069551,16
1,20069944,20091271,2
2,20069944,20081614,11
3,20069944,20267853,9
4,20069944,20089741,11


In [302]:
pwdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410240 entries, 0 to 410239
Data columns (total 3 columns):
sample_1    410240 non-null object
sample_2    410240 non-null object
dist        410240 non-null int64
dtypes: int64(1), object(2)
memory usage: 9.4+ MB


In [303]:
samples = list(set(pwdf.sample_1.tolist()))

In [304]:
len(samples)

641

In [237]:
samples[0:5]

['20282323', '20265597', '20242010BIS', '20066420', '20277326']

In [305]:
wgs = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/WGS_COVID.csv')

In [306]:
wgs.head()

,id,full_date,patient,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
0,20060161,09/03/2020,174,Madrid,H,84,9,3,2020
1,20060178,09/03/2020,175,Madrid,H,76,9,3,2020
2,20062236,10/03/2020,176,Madrid,M,96,10,3,2020
3,20062563,10/03/2020,177,Madrid,H,78,10,3,2020
4,20063795,12/03/2020,178,Madrid,H,85,12,3,2020


In [307]:
wgs.shape

(644, 9)

In [308]:
samples_wgs = wgs.id.tolist()

In [309]:
WGS = set(samples_wgs)
PW = set(samples)

In [310]:
len(PW)

641

In [311]:
missing_wgs = list(PW - WGS)
len(missing_wgs)

119

In [312]:
missing_pw = list(WGS - PW)
len(missing_pw)

122

In [313]:
all_missing = missing_pw + missing_wgs
len(all_missing)

241

In [314]:
wgs.shape

(644, 9)

In [315]:
df = wgs[~wgs.id.isin(missing_pw)]

In [316]:
df.shape

(522, 9)

In [317]:
df2 = pd.DataFrame(missing_wgs, columns=['id'])

In [318]:
df2.head()

,id
0,20372740
1,20104985-8
2,20267853
3,20369536
4,20389458


In [319]:
df.head()

,id,full_date,patient,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year
2,20062236,10/03/2020,176,Madrid,M,96,10,3,2020
3,20062563,10/03/2020,177,Madrid,H,78,10,3,2020
4,20063795,12/03/2020,178,Madrid,H,85,12,3,2020
5,20066054,13/03/2020,179,Madrid,H,90,13,3,2020
6,20066411,15/03/2020,180,Madrid,H,78,15,3,2020


In [321]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 522 entries, 2 to 643
Data columns (total 9 columns):
id                     522 non-null object
full_date              522 non-null object
patient                522 non-null int64
province__autocolor    522 non-null object
SEX__autocolor         522 non-null object
AGE__autocolor         522 non-null object
day                    522 non-null int64
month                  522 non-null int64
year                   522 non-null int64
dtypes: int64(4), object(5)
memory usage: 60.8+ KB


In [322]:
final_df = df.append(df2).reset_index(drop=True)
final_df = final_df[ ['id', 'full_date', 'patient', 'province__autocolor', 'SEX__autocolor',
       'AGE__autocolor', 'day', 'month', 'year'] ]

In [323]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641 entries, 0 to 640
Data columns (total 9 columns):
id                     641 non-null object
full_date              522 non-null object
patient                522 non-null float64
province__autocolor    522 non-null object
SEX__autocolor         522 non-null object
AGE__autocolor         522 non-null object
day                    522 non-null float64
month                  522 non-null float64
year                   522 non-null float64
dtypes: float64(4), object(5)
memory usage: 45.2+ KB


In [324]:
metadata = pd.read_csv('/home/laura/ANALYSIS/MISC/microreact/wgs_covid_metadata.csv')

In [330]:
metadata.head()

,id,DEPARTMENT__autocolor
0,20358933,Digestivo
1,20360682,Digestivo
2,20367276,Digestivo
3,20365371,Digestivo
4,2143593,Otorrino


In [331]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
id                       101 non-null object
DEPARTMENT__autocolor    101 non-null object
dtypes: object(2)
memory usage: 1.7+ KB


In [332]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641 entries, 0 to 640
Data columns (total 9 columns):
id                     641 non-null object
full_date              522 non-null object
patient                522 non-null float64
province__autocolor    522 non-null object
SEX__autocolor         522 non-null object
AGE__autocolor         522 non-null object
day                    522 non-null float64
month                  522 non-null float64
year                   522 non-null float64
dtypes: float64(4), object(5)
memory usage: 45.2+ KB


In [333]:
final_df = final_df.fillna(0)
final_df.day = final_df.day.astype('Int64')
final_df.month = final_df.month.astype('Int64')
final_df.year = final_df.year.astype('Int64')
final_df.replace(0, np.nan, inplace=True)

In [334]:
final_df = final_df.merge(metadata, how='outer')

In [335]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 649 entries, 0 to 648
Data columns (total 10 columns):
id                       649 non-null object
full_date                522 non-null object
patient                  522 non-null float64
province__autocolor      522 non-null object
SEX__autocolor           522 non-null object
AGE__autocolor           522 non-null object
day                      522 non-null Int64
month                    522 non-null Int64
year                     522 non-null Int64
DEPARTMENT__autocolor    101 non-null object
dtypes: Int64(3), float64(1), object(6)
memory usage: 77.7+ KB


In [336]:
final_df.head()

,id,full_date,patient,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor
0,20062236,10/03/2020,176.0,Madrid,M,96,10,3,2020,NaN
1,20062563,10/03/2020,177.0,Madrid,H,78,10,3,2020,NaN
2,20063795,12/03/2020,178.0,Madrid,H,85,12,3,2020,NaN
3,20066054,13/03/2020,179.0,Madrid,H,90,13,3,2020,NaN
4,20066411,15/03/2020,180.0,Madrid,H,78,15,3,2020,NaN


In [337]:
def extract_lineage(sample):
    pango_file = os.path.join('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Annotation/pangolin', sample + ".lineage.csv")
    if os.path.exists(pango_file):
        df = pd.read_csv(pango_file)
        return df.lineage[0]
    else:
        return None

In [338]:
final_df['Lineage__autocolor'] = final_df.apply(lambda x: extract_lineage(x['id']), axis=1)

In [339]:
query_dario = [20242010,20242093,20242702,20247604,20247878,20250481,20250983,20253374,20253475,20253522,20253595,20255027,20255086,20255313,20255524,20255685,20255915,20255928,20258070,20258214,20258238,20258322,20258440,20261662,20263252,20265521,20267856,20269026,20269341,20269791,20272101,20272242,20273792,20274440,20277245,20277326,20279174,20279180,20279189,20279243,20279283,20286667,20286864,20286872,20288207,20289563,20303453]
query_dario = [str(x) for x in query_dario]


In [343]:
query_dario_2 = [20265521,20269791,20272101,20274440,20277245,20277326,20279174,20279180,20279189]
query_dario_2 = [str(x) for x in query_dario_2]

In [340]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 649 entries, 0 to 648
Data columns (total 11 columns):
id                       649 non-null object
full_date                522 non-null object
patient                  522 non-null float64
province__autocolor      522 non-null object
SEX__autocolor           522 non-null object
AGE__autocolor           522 non-null object
day                      522 non-null Int64
month                    522 non-null Int64
year                     522 non-null Int64
DEPARTMENT__autocolor    101 non-null object
Lineage__autocolor       641 non-null object
dtypes: Int64(3), float64(1), object(7)
memory usage: 82.7+ KB


In [346]:
final_df['Dario__autocolor'] = final_df.apply(lambda x: x['id'] in query_dario, axis=1)
final_df['Dario2__autocolor'] = final_df.apply(lambda x: x['id'] in query_dario_2, axis=1)

In [347]:
final_df.to_csv('/home/laura/ANALYSIS/VARIANT_CALLING/COVID/Compare/2020-10-18_Compare_rush/wgs.csv', index=False)

In [349]:
final_df[final_df.id == '20274440']

,id,full_date,patient,province__autocolor,SEX__autocolor,AGE__autocolor,day,month,year,DEPARTMENT__autocolor,Lineage__autocolor,Dario__autocolor,Dario2__autocolor
354,20274440,17/08/2020,422.0,Madrid,F,30,17,8,2020,Geriatria,B.1,True,True


In [301]:
final_df.shape

(709, 12)